In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import sys
import os
from pytube import YouTube
from moviepy.editor import *
from pydub import AudioSegment

from youtubesearchpython import VideosSearch

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install youtube_dl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 37.7 MB/s eta 0:00:0000:01


In [3]:
!pip install youtube-search-python 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.5/99.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 5.9 MB/s eta 0:00:00


In [4]:
!pip install pytube


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.3 MB/s eta 0:00:00


In [5]:
!pip install moviepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 64.6 MB/s eta 0:00:00:00:0100:01
  Created wheel for moviepy: filename=moviepy-1.0.3-py3-none-any.whl size=110720 sha256=683a5c0d8e869d54de87312cb9ec4322f2b4161a580254af826a37a0159d361c
  Stored in directory: /root/.cache/pip/wheels/96/32/2d/e10123bd88fbfc02fed53cc18c80a171d3c87479ed845fa7c1
Successfully built moviepy
  Attempting uninstall: decorator
    Found existing installation: decorator 5.1.1
    Uninstalling decorator-5.1.1:
      Successfully uninstalled decorator-5.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2023.12.2.post1 requires fsspec==2023.12.2, but you have fsspec 2024.2.0 which is incompatible.


In [6]:
!pip install pydub

In [8]:
def download_videos(singer_name, num_videos):
    print(f"Searching and downloading {num_videos} videos of {singer_name} from YouTube...")

    # Use youtube-search-python to search for videos by singer name
    search = VideosSearch(singer_name, limit=num_videos)
    search_results = search.result()

    for i, video_info in enumerate(search_results["result"], start=1):
        video_url = f"https://www.youtube.com/watch?v={video_info['id']}"
        try:
            yt_video = YouTube(video_url)
            # Choose the mp4 stream
            video_stream = yt_video.streams.filter(file_extension='mp4', progressive=True).first()
            video_stream.download(output_path=os.getcwd(), filename=f"{singer_name}_{i}.mp4")
            print(f"Video {i} downloaded successfully.")
        except Exception as e:
            print(f"Error downloading video {i}: {e}")

    print("Download completed.")


In [9]:
def convert_to_audio(video_path):
    print(f"Converting {video_path} to audio...")
    video = VideoFileClip(video_path)
    video.audio.write_audiofile(video_path.replace(".mp4", ".mp3"))


In [10]:
def cut_audio(audio_path, duration):
    print(f"Cutting first {duration} seconds from {audio_path}...")
    audio = AudioSegment.from_file(audio_path)
    cut_audio = audio[duration * 1000:]
    cut_audio.export(audio_path.replace(".mp3", "_cut.mp3"), format="mp3")


In [11]:
def merge_audios(audio_list, output_file):
    print("Merging audio files...")
    combined_audio = AudioSegment.silent(duration=0)
    for audio_file in audio_list:
        audio = AudioSegment.from_file(audio_file)
        combined_audio += audio

    combined_audio.export(output_file, format="mp3")

In [12]:
def main():
    # Check the number of command line arguments
#     print(sys.argv)
#     print("Received command line arguments:", sys.argv)


#     if len(sys.argv) != 5:
#         print("Usage: python <program.py> <SingerName> <NumberOfVideos> <AudioDuration> <OutputFileName>")
#         sys.exit(1)

#     singer_name = sys.argv[1]
#     num_videos = int(sys.argv[2])
#     audio_duration = int(sys.argv[3])
#     output_file = sys.argv[4]

    try:
        print(singer_name)
        download_videos(singer_name, num_videos)

        # Convert videos to audio
        for i in range(1, num_videos + 1):
            video_path = f"{singer_name}_{i}.mp4"
            convert_to_audio(video_path)

        # Cut first Y seconds from all audio files
        for i in range(1, num_videos + 1):
            audio_path = f"{singer_name}_{i}.mp3"
            cut_audio(audio_path, audio_duration)

        # Merge all audios
        audio_list = [f"{singer_name}_{i}_cut.mp3" for i in range(1, num_videos + 1)]
        merge_audios(audio_list, output_file)

        print("Mashup created successfully!")

    except Exception as e:
        print(f"An error occurred: {e}")
        sys.exit(1)

        
if __name__ == "__main__":
    
    singer_name = "Arjan Dhillon"
    num_videos = 4
    audio_duration = 25
    output_file = "Final.mp3"
    main()
    


Arjan Dhillon
Searching and downloading 4 videos of Arjan Dhillon from YouTube...
Video 1 downloaded successfully.
Video 2 downloaded successfully.
Video 3 downloaded successfully.
Video 4 downloaded successfully.
Download completed.
Converting Arjan Dhillon_1.mp4 to audio...
MoviePy - Writing audio in Arjan Dhillon_1.mp3


MoviePy - Done.
Converting Arjan Dhillon_2.mp4 to audio...
MoviePy - Writing audio in Arjan Dhillon_2.mp3


MoviePy - Done.
Converting Arjan Dhillon_3.mp4 to audio...
MoviePy - Writing audio in Arjan Dhillon_3.mp3


MoviePy - Done.
Converting Arjan Dhillon_4.mp4 to audio...
MoviePy - Writing audio in Arjan Dhillon_4.mp3


MoviePy - Done.
Cutting first 25 seconds from Arjan Dhillon_1.mp3...
Cutting first 25 seconds from Arjan Dhillon_2.mp3...
Cutting first 25 seconds from Arjan Dhillon_3.mp3...
Cutting first 25 seconds from Arjan Dhillon_4.mp3...
Merging audio files...
Mashup created successfully!
